In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mojtabashahi/persian-miniature")

print("Path to dataset files:", path)

100%|██████████| 621M/621M [00:07<00:00, 89.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mojtabashahi/persian-miniature/versions/1


In [10]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [11]:

dataset_dir = '/root/.cache/kagglehub/datasets/mojtabashahi/persian-miniature/versions/1'

img_height, img_width = 128, 128
batch_size = 32


In [12]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 713 images belonging to 5 classes.
Found 177 images belonging to 5 classes.


In [13]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # خروجی چندکلاسه
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,285 (12.61 MB)

 Trainable params: 3,305,285 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:

epochs = 15
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)



Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.2346 - loss: 1.6710 - val_accuracy: 0.4068 - val_loss: 1.2694
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.3745 - loss: 1.3884 - val_accuracy: 0.4746 - val_loss: 1.1330
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5119 - loss: 1.2001 - val_accuracy: 0.4576 - val_loss: 1.1542
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5186 - loss: 1.1120 - val_accuracy: 0.6780 - val_loss: 0.9231
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5965 - loss: 1.0107 - val_accuracy: 0.7232 - val_loss: 0.8323
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.6808 - loss: 0.8710 - val_accuracy: 0.8249 - val_loss: 0.5815
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7056 - loss: 0.7537 - val_accuracy: 0.8192 - val_loss: 0.5553
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.7343 - loss: 0.6493 - val_accuracy: 0.8249 - val_loss:

In [16]:

val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8030 - loss: 0.5313
Validation Loss: 0.5120
Validation Accuracy: 0.8079


In [17]:

model.save('persian_miniature_cnn_model.h5')
print("Model saved successfully!")




Model saved successfully!
